## 读入分析数据

In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.impute import SimpleImputer as Imputer
from sklearn.linear_model import LogisticRegression

# File system manangement
import os

import joblib
import requests
from time import time

In [2]:
# List files available
print(os.listdir("./input/"))

['application_test.csv', 'application_train.csv', 'bureau.csv', 'bureau_balance.csv', 'credit_card_balance.csv', 'HomeCredit_columns_description.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'previous_application.csv', 'sample_submission.csv']


In [3]:
# Training data
app_train = pd.read_csv('./input/application_train.csv')
app_test = pd.read_csv('./input/application_test.csv')
bureau = pd.read_csv('./input/bureau.csv')
bb = pd.read_csv('./input/bureau_balance.csv')
prev = pd.read_csv('./input/previous_application.csv')
cc = pd.read_csv('./input/credit_card_balance.csv')
ins = pd.read_csv('./input/installments_payments.csv')
pos = pd.read_csv('./input/POS_CASH_balance.csv')

print('Training data shape: ', app_train.shape)
app_train.head()

Training data shape:  (307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
train_labels = app_train['TARGET']

app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 121)
Testing Features shape:  (48744, 121)


In [5]:
# numeric_columns = []
# category_columns = []

# for col in app_train:

#     if app_train[col].isna().sum() > 0.7 * app_train.shape[0]:
#         continue

#     if app_train[col].dtype == ('object') or app_train[col].dtype == ('bool'):
#         category_columns.append(col)
#     else:
#         numeric_columns.append(col)

# print("Number of categorical feature:", len(category_columns))
# print("Number of numerical feature:", len(numeric_columns))
# print("Number of missing feature:", app_train.shape[1] - len(category_columns) - len(numeric_columns))

## 预处理

In [11]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if ((df[col].dtype == 'object') or (df[col].dtype == 'bool'))]
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [12]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
 
class appTrainTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        # DAYS_EMPLOYED_anom
        X['DAYS_EMPLOYED_anom'] = (X['DAYS_EMPLOYED'] == 365243)
        X['DAYS_EMPLOYED'].replace(365243, np.nan, inplace = True)

        # Some simple new features (percentages)
        X['DAYS_EMPLOYED_PERC'] = X['DAYS_EMPLOYED'] / X['DAYS_BIRTH']
        X['INCOME_CREDIT_PERC'] = X['AMT_INCOME_TOTAL'] / X['AMT_CREDIT']
        X['INCOME_PER_PERSON'] = X['AMT_INCOME_TOTAL'] / X['CNT_FAM_MEMBERS']
        X['ANNUITY_INCOME_PERC'] = X['AMT_ANNUITY'] / X['AMT_INCOME_TOTAL']
        X['PAYMENT_RATE'] = X['AMT_ANNUITY'] / X['AMT_CREDIT']
        X.replace(np.inf, np.nan, inplace = True)

        return X

class dropTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, rate):
        self.rate = rate
        self.drop = []
        return None
 
    def fit(self, X, y=None):
        drop = []
        for col in X:
            if X[col].isna().sum() > self.rate * X.shape[0]:
                drop.append(col)
        self.drop = drop
        return self
 
    def transform(self, X_copy, y=None):
        X_copy.drop(self.drop, axis = 1, inplace = True)
        return X_copy

class bureauTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, bureau, bb):
        self.bb = bb
        self.bureau = bureau
        return None
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X_copy, y=None):
        # app_train treatment
        bb_copy, bb_cat = one_hot_encoder(self.bb, True)
        bureau_copy, bureau_cat = one_hot_encoder(self.bureau, True)

        # bb Treatment
        bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
        for col in bb_cat:
            bb_aggregations[col] = ['mean']
        bb_agg = bb_copy.groupby('SK_ID_BUREAU').agg(bb_aggregations)
        bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
        bureau_copy = bureau_copy.join(bb_agg, how='left', on='SK_ID_BUREAU')
        bureau_copy.drop(['SK_ID_BUREAU'], axis=1, inplace= True)

        ## bureau Treatment
        # Bureau and bureau_balance numeric features
        num_aggregations = {
            'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
            'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
            'DAYS_CREDIT_UPDATE': ['mean'],
            'CREDIT_DAY_OVERDUE': ['max', 'mean'],
            'AMT_CREDIT_MAX_OVERDUE': ['mean'],
            'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
            'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
            'AMT_CREDIT_SUM_OVERDUE': ['mean'],
            'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
            'AMT_ANNUITY': ['max', 'mean'],
            'CNT_CREDIT_PROLONG': ['sum'],
            'MONTHS_BALANCE_MIN': ['min'],
            'MONTHS_BALANCE_MAX': ['max'],
            'MONTHS_BALANCE_SIZE': ['mean', 'sum']
        }

        # Bureau and bureau_balance categorical features
        cat_aggregations = {}
        for cat in bureau_cat: 
            cat_aggregations[cat] = ['mean']
        for cat in bb_cat: 
            cat_aggregations[cat + "_MEAN"] = ['mean']

        bureau_agg = bureau_copy.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
        bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])

        # Bureau: Active credits - using only numerical aggregations
        active = bureau_copy[bureau_copy['CREDIT_ACTIVE_Active'] == 1]
        active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
        active_agg.columns = pd.Index(['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
        bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')


        # Bureau: Closed credits - using only numerical aggregations
        closed = bureau_copy[bureau_copy['CREDIT_ACTIVE_Closed'] == 1]
        closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
        closed_agg.columns = pd.Index(['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
        bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')

        return X_copy

class previousTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, prev):
        self.prev = prev
        return None
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):

        prev, cat_cols = one_hot_encoder(self.prev, nan_as_category= True)
        # Days 365.243 values -> nan
        prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
        prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
        prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
        prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
        prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)

        # Add feature: value ask / value received percentage
        prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
        # Previous applications numeric features
        num_aggregations = {
            'AMT_ANNUITY': ['min', 'max', 'mean'],
            'AMT_APPLICATION': ['min', 'max', 'mean'],
            'AMT_CREDIT': ['min', 'max', 'mean'],
            'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
            'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
            'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
            'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
            'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
            'DAYS_DECISION': ['min', 'max', 'mean'],
            'CNT_PAYMENT': ['mean', 'sum'],
        }
        # Previous applications categorical features
        cat_aggregations = {}
        for cat in cat_cols:
            cat_aggregations[cat] = ['mean']
        
        prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
        prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
        # Previous Applications: Approved Applications - only numerical features
        approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
        approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
        approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
        prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
        # Previous Applications: Refused Applications - only numerical features
        refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
        refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
        refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
        prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')

        X = X.join(prev_agg, how='left', on='SK_ID_CURR')
        return X

class posTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, pos):
        self.pos = pos
        return None
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):

        pos, cat_cols = one_hot_encoder(self.pos, True)
        # Features
        aggregations = {
            'MONTHS_BALANCE': ['max', 'mean', 'size'],
            'SK_DPD': ['max', 'mean'],
            'SK_DPD_DEF': ['max', 'mean']
        }
        for cat in cat_cols:
            aggregations[cat] = ['mean']
        
        pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
        pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
        # Count pos cash accounts
        pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()

        X = X.join(pos_agg, how='left', on='SK_ID_CURR')
        return X
    
class installmentsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, ins):
        self.ins = ins
        return None
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        ins, cat_cols = one_hot_encoder(self.ins, True)

        # Percentage and difference paid in each installment (amount paid and installment value)
        ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
        ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
        # Days past due and days before due (no negative values)
        ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
        ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
        ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
        ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
        # Features: Perform aggregations
        aggregations = {
            'NUM_INSTALMENT_VERSION': ['nunique'],
            'DPD': ['max', 'mean', 'sum'],
            'DBD': ['max', 'mean', 'sum'],
            'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
            'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
            'AMT_INSTALMENT': ['max', 'mean', 'sum'],
            'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
            'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
        }
        for cat in cat_cols:
            aggregations[cat] = ['mean']
        ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
        ins_agg.columns = pd.Index(['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
        # Count installments accounts
        ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()

        X = X.join(ins_agg, how='left', on='SK_ID_CURR')
        return X

class ccTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, cc):
        self.cc = cc
        return None
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        cc, cat_cols = one_hot_encoder(self.cc, True)
        # General aggregations
        cc.drop(['SK_ID_PREV'], axis= 1, inplace = True)
        cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
        cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
        # Count credit card lines
        cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
        
        X = X.join(cc_agg, how='left', on='SK_ID_CURR')
        return X

In [13]:
a = bureauTransformer(bureau, bb)
c = previousTransformer(prev)
b = appTrainTransformer()
d = posTransformer(pos)
e = ccTransformer(cc)
f = installmentsTransformer(ins)
X = app_train.copy()
g = dropTransformer(0.7)

X = g.fit_transform(X)
X = a.fit_transform(X)
X = c.fit_transform(X)
X = f.fit_transform(X)
X = d.fit_transform(X)
X = e.fit_transform(X)
X = b.fit_transform(X)

numeric_columns = []
category_columns = []

for col in X:
    
    if X[col].dtype == ('object') or X[col].dtype == ('bool'):
        category_columns.append(col)
    else:
        numeric_columns.append(col)

print("Number of categorical feature:", len(category_columns))
print("Number of numerical feature:", len(numeric_columns))

Number of categorical feature: 17
Number of numerical feature: 544


In [14]:
numeric_transformer = Pipeline(steps=[
    ('impute_nan', Imputer(missing_values=np.nan,strategy='median')),
    ('scaler', StandardScaler())
])

category_transformer = Pipeline(steps=[
    ('impute', Imputer(strategy='constant', fill_value='missing')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
])

preprocesser = ColumnTransformer(transformers=[
    ('numeric', numeric_transformer, numeric_columns),
    ('category', category_transformer, category_columns)
]) 

In [15]:
pipeline_pre = Pipeline(steps=[
    ('drop', dropTransformer(0.7))
    ('bureau_and_bureaubalance', bureauTransformer(bureau, bb)),
    ('previous_application', previousTransformer(prev)),
    ('pos', posTransformer(pos)),
    ('cc', ccTransformer(cc)),
    ('installments',  installmentsTransformer(ins)),
    ('app_train',  appTrainTransformer()),
    ('preprocesser', preprocesser)
])

## 建造Pipeline

In [44]:
from sklearn.model_selection import KFold
import lightgbm as lgb

class kfoldlgb(BaseEstimator, TransformerMixin):
    def __init__(self, k):
        self.k = k
        self.clf = []
        return None
 
    def fit(self, X, y=None):
        kf = KFold(n_splits=self.k)
        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            gbm = lgb.LGBMClassifier(
                nthread=4,
                n_estimators=100,
                learning_rate=0.02,
                num_leaves=34,
                colsample_bytree=0.9497036,
                subsample=0.8715623,
                max_depth=8,
                reg_alpha=0.041545473,
                reg_lambda=0.0735294,
                min_split_gain=0.0222415,
                min_child_weight=39.3259775,
                silent=-1,
                verbose=-1, )
            gbm.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric= 'auc', verbose= 50, early_stopping_rounds= 200)
            self.clf.append(gbm)
        return self
 
    def predict_proba(self, X, y=None):
        for i in range(self.k):
            tmp = self.clf[i].predict_proba(X)
            if i!=0:
                result = result + tmp/self.k
            else:
                result = tmp
        return result

In [30]:
app_pre_train = pipeline_pre.fit_transform(app_train)
app_pre_test = pipeline_pre.transform(app_test)

In [45]:
# 使用sklearn的pipelines训练一个模型
clf = kfoldlgb(2)
clf.fit(app_pre_train, train_labels)

Training until validation scores don't improve for 200 rounds
[50]	training's auc: 0.758115	training's binary_logloss: 0.251934	valid_1's auc: 0.740073	valid_1's binary_logloss: 0.258609
[100]	training's auc: 0.779321	training's binary_logloss: 0.242135	valid_1's auc: 0.756433	valid_1's binary_logloss: 0.250983
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.779321	training's binary_logloss: 0.242135	valid_1's auc: 0.756433	valid_1's binary_logloss: 0.250983
Training until validation scores don't improve for 200 rounds
[50]	training's auc: 0.759508	training's binary_logloss: 0.255326	valid_1's auc: 0.743862	valid_1's binary_logloss: 0.254888
[100]	training's auc: 0.780319	training's binary_logloss: 0.245259	valid_1's auc: 0.757487	valid_1's binary_logloss: 0.247558
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.780319	training's binary_logloss: 0.245259	valid_1's auc: 0.757487	valid_1's binary_logloss: 0.247558


kfoldlgb(k=2)

## Kaggle 输出 （用于上传） 

In [46]:
Y = clf.predict_proba(app_pre_test)

In [47]:
log_reg_pred = Y[:, 1]
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = log_reg_pred
submit.to_csv('baseline.csv', index = False)

## Docker API

In [49]:
joblib.dump(pipeline_pre, './docker/preprocess_model.joblib')
joblib.dump(clf, './docker/clf.joblib')

['./docker/clf.joblib']

In [79]:
!docker build --tag project_2 ./docker/

#2 [internal] load .dockerignore
#2 transferring context: 2B done
#2 DONE 0.0s

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 531B 0.0s done
#1 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.7-slim
#3 DONE 4.3s

#4 [1/12] FROM docker.io/library/python:3.7-slim@sha256:ac0c17ada735be16aae...
#4 DONE 0.0s

#5 [2/12] WORKDIR /usr/src/app
#5 CACHED

#7 [internal] load build context
#7 transferring context: 362B done
#7 DONE 0.0s

#6 [3/12] RUN pip install -i http://pypi.douban.com/simple/ --trusted-host=...
#6 1.155 Looking in indexes: http://pypi.douban.com/simple/
#6 1.308 Collecting flask
#6 1.321   Downloading http://pypi.doubanio.com/packages/f2/28/2a03252dfb9ebf377f40fba6a7841b47083260bf8bd8e737b0c6952df83f/Flask-1.1.2-py2.py3-none-any.whl (94 kB)
#6 1.440 Collecting gunicorn
#6 1.457   Downloading http://pypi.doubanio.com/packages/69/ca/926f7cd3a2014b16870086b2d0fdc84a9e49473c68a8dff8b57f7c156f43/gunicorn-20.0.4-py2.py3-none-an

In [80]:
!docker run --name project_2 -p 8080:8080 -d project_2

0ee71faa5204b6b9cdfa76512c976b0df8cf20ca609025d6a6f6473c82687d24


In [12]:
# 测试性能
start = time()
app_test = app_test[:1000]
# for _, row in tqdm(test_data.iterrows(), total=test_data.shape[0]):
for _, row in app_test.iterrows():
    item = row.to_dict()
    res = requests.post('http://localhost:8080',json = item) ## json 为 dict
    print(res.text)
end = time()
'Used', end-start, 'seconds'

{"predict":0.943288588760659}

{"predict":0.849130012207588}

{"predict":0.9587926927194563}

{"predict":0.9565269003869907}

{"predict":0.8618659445891443}

{"predict":0.9557973205602237}

{"predict":0.9575213771704587}

{"predict":0.9043963531094006}

{"predict":0.9728694348879142}

{"predict":0.8465245919723765}

{"predict":0.9312044524238939}

{"predict":0.9078466600404103}

{"predict":0.8651853084394927}

{"predict":0.8725078629213989}

{"predict":0.9087002582073126}

{"predict":0.8980610342898866}

{"predict":0.9149288283127897}

{"predict":0.9609107727455217}

{"predict":0.9075313268921259}

{"predict":0.94733270303698}

{"predict":0.9643455270143779}

{"predict":0.9771563460348374}

{"predict":0.9662766424091817}

{"predict":0.9372181091880096}

{"predict":0.9515028073058066}

{"predict":0.8752548789353427}

{"predict":0.9047274969679375}

{"predict":0.9348449009836186}

{"predict":0.9474752976536803}

{"predict":0.9058484218716427}

{"predict":0.9672326338206169}

{"predict":0

{"predict":0.965294756423321}

{"predict":0.9272180630955329}

{"predict":0.9618582501932241}

{"predict":0.9810181065917198}

{"predict":0.8946016722630908}

{"predict":0.9048992341220032}

{"predict":0.9800110706274484}

{"predict":0.8609083449227882}

{"predict":0.8896015798249867}

{"predict":0.9613186603085153}

{"predict":0.96521096432686}

{"predict":0.9155772824540835}

{"predict":0.9664735997929373}

{"predict":0.9010813975536165}

{"predict":0.7481187650699274}

{"predict":0.9459643328626435}

{"predict":0.943188702777191}

{"predict":0.9574839867416047}

{"predict":0.8448623621450808}

{"predict":0.9450066375916505}

{"predict":0.9486523504000168}

{"predict":0.8659053782556867}

{"predict":0.9100058634721183}

{"predict":0.9505900061976997}

{"predict":0.7481754660406988}

{"predict":0.9751795233160904}

{"predict":0.8597684150942446}

{"predict":0.9769470538968962}

{"predict":0.8466603664261044}

{"predict":0.9768369655642299}

{"predict":0.9641544342206475}

{"predict":0

{"predict":0.9619149659929298}

{"predict":0.9369426133269232}

{"predict":0.9504382392436113}

{"predict":0.9274819435274702}

{"predict":0.9193348987916362}

{"predict":0.9446794440880667}

{"predict":0.9710624753441405}

{"predict":0.9333184472719177}

{"predict":0.8756260072862534}

{"predict":0.9717181629161336}

{"predict":0.9297084241613732}

{"predict":0.8462895342313903}

{"predict":0.977087601294427}

{"predict":0.9308780182475048}

{"predict":0.9222539264803836}

{"predict":0.9723514184537055}

{"predict":0.9243857732129463}

{"predict":0.8843482901929973}

{"predict":0.9311137546519735}

{"predict":0.7563867325578781}

{"predict":0.952573828034229}

{"predict":0.9679674292967835}

{"predict":0.7817524818330956}

{"predict":0.9707672219717102}

{"predict":0.8917053040612871}

{"predict":0.8578692358342025}

{"predict":0.9223763063620741}

{"predict":0.9790459357609841}

{"predict":0.8109639080706766}

{"predict":0.8169331714442732}

{"predict":0.9773319730135788}

{"predict"

{"predict":0.8468590808918675}

{"predict":0.9057110635013312}

{"predict":0.8973447932455028}

{"predict":0.9539681904107633}

{"predict":0.9565202306418036}

{"predict":0.9148682595608231}

{"predict":0.9173599861046347}

{"predict":0.8884300532987462}

{"predict":0.9313314782630521}

{"predict":0.977529846132442}

{"predict":0.7980851439898894}

{"predict":0.9531781341229536}

{"predict":0.9234622184940284}

{"predict":0.917879410678432}

{"predict":0.9798317734044452}

{"predict":0.9588077758713529}

{"predict":0.9808062350145088}

{"predict":0.7581202111523608}

{"predict":0.9243872162095577}

{"predict":0.9018059234227283}

{"predict":0.9555485993542461}

{"predict":0.9403699097901975}

{"predict":0.9257162290796035}

{"predict":0.8949452974850792}

{"predict":0.9704856324447488}

{"predict":0.9680445189994286}

{"predict":0.9636145278471802}

{"predict":0.8772429643361711}

{"predict":0.8834299344665912}

{"predict":0.9184449083225488}

{"predict":0.9592203422183928}

{"predict"

('Used', 19.39782190322876, 'seconds')